In [15]:
#Import inputs

import pandas as pd

# Specify the file path and sheet name
file_path_1 = r'input\certifications_additional_metadata_INPUT.xlsx'
sheet_name_1 = 'Certificados'  # replace with the name of the sheet you want to read
# Read the sheet into a DataFrame
df_certif_adj = pd.read_excel(file_path_1, sheet_name=sheet_name_1, engine='openpyxl')
# Display the first few rows of the DataFrame

file_path_2 = r'input\email.xlsx'
sheet_name_2 = 'Plan de recursos'  # replace with the name of the sheet you want to read
# Read the sheet into a DataFrame
df_email = pd.read_excel(file_path_2, sheet_name=sheet_name_2, engine='openpyxl')

file_path_3 = r'input\export_certifications.xlsx'
sheet_name_3 = 'Certificados'  # replace with the name of the sheet you want to read
# Read the sheet into a DataFrame
df_exp_cert = pd.read_excel(file_path_3, sheet_name=sheet_name_3, engine='openpyxl')

print(df_certif_adj.columns)
print(df_email.columns)

print(df_exp_cert.columns)

Index(['EMPLOYEE CODE', 'NAME', 'SURNAME', 'ORGANIZATIONAL UNIT',
       'MANAGEMENT UNIT', 'CERTIFICATION', 'COMMENTS', 'CERTIFYING ENTITY',
       'ISSUE DATE', 'VALIDITY DATE MODIFIED', 'NORMALIZED CERTIFICATION',
       'EMAIL'],
      dtype='object')
Index(['Profess.', 'Name of the professional', 'Email', 'OU of the resource'], dtype='object')
Index(['EMPLOYEE CODE', 'NAME', 'SURNAME', 'ORGANIZATIONAL UNIT',
       'MANAGEMENT UNIT', 'CERTIFICATION', 'COMMENTS', 'CERTIFYING ENTITY',
       'ISSUE DATE', 'VALIDITY DATE'],
      dtype='object')


In [16]:
#Join all inputs

df_certif_adj = df_certif_adj[['EMPLOYEE CODE','VALIDITY DATE MODIFIED']]
merged_df_valid = pd.merge(df_exp_cert, df_certif_adj, on='EMPLOYEE CODE', how='left')  
merged_df_valid.drop(['VALIDITY DATE'], axis=1, inplace=True)
print(merged_df_valid)

df_email = df_email[['Profess.','Email']]
df_email.rename(columns={'Profess.': 'EMPLOYEE CODE'}, inplace=True)
merged_df_valid_email = pd.merge(merged_df_valid, df_email, on='EMPLOYEE CODE', how='left')  



      EMPLOYEE CODE      NAME     SURNAME         ORGANIZATIONAL UNIT  \
0            451927    ANDREA       CENSI     DBT I&D Data Management   
1            451927    ANDREA       CENSI     DBT I&D Data Management   
2            451927    ANDREA       CENSI     DBT I&D Data Management   
3            451927    ANDREA       CENSI     DBT I&D Data Management   
4            451927    ANDREA       CENSI     DBT I&D Data Management   
...             ...       ...         ...                         ...   
4468         470297   ALFREDO     SIRLETO  DBT I&D Advanced Analytics   
4469         470297   ALFREDO     SIRLETO  DBT I&D Advanced Analytics   
4470         470297   ALFREDO     SIRLETO  DBT I&D Advanced Analytics   
4471         470297   ALFREDO     SIRLETO  DBT I&D Advanced Analytics   
4472         576483  PATRIZIO  DI PASTENA  DBT I&D Advanced Analytics   

                          MANAGEMENT UNIT  \
0     Digital Business Technologies (DBT)   
1     Digital Business Technologi

In [3]:
#Normalize certifications
from langchain import PromptTemplate, LLMChain
from langchain.llms import OpenAI
import os
 
# Initialize envirnonment variable OPENAI_API_KEY with your API key 
with open('api_key.txt', 'r') as file:
    # Read the first line
    os.environ['OPENAI_API_KEY'] = file.readline().strip()  

# Template for the prompt
template = """Question: {question}

Answer: """

prompt = PromptTemplate(template=template,input_variables=['question'])


# OpenAI model
model = OpenAI(model_name="gpt-3.5-turbo-16k",temperature=0)


# LLMChain
llm_chain = LLMChain(
prompt=prompt,
llm=model
)

distinct_values_certifications = merged_df_valid_email['CERTIFICATION'].unique()

cert_list_str=''
for elem in distinct_values_certifications:
    cert_list_str=cert_list_str + str(elem) + '\n'

question=""" 

Under the '***' marker, you'll find a collection of technical certifications.

Some of these certifications have different text representations, but they essentially represent the same qualifications.

First, identify certifications with multiple names.

Next, standardize the names by assigning a unique one to each.

Record the results in a CSV file using a semicolon as the delimiter, including two fields:

The original certification name
The standardized certification name

***
"""+cert_list_str
#print(question)


# Run the model
answer = llm_chain.run(question)


C:\Users\aiannini\AppData\Roaming\Python\Python311\site-packages\langchain\llms\openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
C:\Users\aiannini\AppData\Roaming\Python\Python311\site-packages\langchain\llms\openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [4]:
import csv 

# Split the input string into lines
lines = answer.strip().split('\n')

# Specify the output file path (e.g., 'output.csv')
output_file_path = 'certifications_normalized.csv'

# Write the content to a CSV file with a semicolon delimiter
with open(output_file_path, 'w', newline='') as outfile:
    csv_writer = csv.writer(outfile, delimiter=';')  # Specify the delimiter
    for line in lines:
        row = line.split(';')
        csv_writer.writerow(row)
        

In [20]:
import pandas as pd
from io import StringIO
 
# Create a DataFrame from the openai answer
df_normalized_certifications = pd.read_csv(StringIO(answer), sep=';') 

df_output = pd.merge(merged_df_valid_email, df_normalized_certifications, left_on='CERTIFICATION', right_on='Original Certification Name', how='inner')

df_output = df_output.rename(columns={' Standardized Certification Name': 'NORMALIZED CERTIFICATION'})

df_output.loc[df_output['CERTIFYING ENTITY'] == 'Tibco Software', 'NORMALIZED CERTIFICATION'] = df_output['CERTIFICATION']

df_output = df_output[['EMPLOYEE CODE','NAME','SURNAME','ORGANIZATIONAL UNIT','MANAGEMENT UNIT','CERTIFICATION',
                       'COMMENTS','CERTIFYING ENTITY','ISSUE DATE','VALIDITY DATE MODIFIED','NORMALIZED CERTIFICATION']]

df_output['NORMALIZED CERTIFICATION'] = df_output['NORMALIZED CERTIFICATION'].str.lstrip()

df_output = df_output.drop_duplicates(subset=['EMPLOYEE CODE', 'NORMALIZED CERTIFICATION'])

df_output.to_excel('output/certifications_additional_metadata_OUTPUT_IANNINI.xlsx',index = False)